In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
area_pre_feature_selection = pd.read_csv('../../data/pre_training/area_pre_feature_selection.csv')
district_pre_feature_selection = pd.read_csv('../../data/pre_training/district_pre_feature_selection.csv')

In [9]:
area_features = area_pre_feature_selection.drop('area_crimes_this_hour', axis=1)
district_features = district_pre_feature_selection.drop('district_crimes_this_hour', axis=1)

area_target = area_pre_feature_selection[['year', 'area_crimes_this_hour']]
district_target = district_pre_feature_selection[['year', 'district_crimes_this_hour']]

In [10]:
# break the area dataset into testing and training datasets
area_feature_training_data = area_features[area_features['year'] < 2020].reset_index(drop=True)
area_feature_testing_data = area_features[area_features['year'] == 2020].reset_index(drop=True)

area_target_training_data = area_target[area_target['year'] < 2020].reset_index(drop=True)
area_target_testing_data = area_target[area_target['year'] == 2020].reset_index(drop=True)

In [11]:
# break the district dataset into testing and training datasets
district_feature_training_data = district_features[district_features['year'] < 2020].reset_index(drop=True)
district_feature_testing_data = district_features[district_features['year'] == 2020].reset_index(drop=True)

district_target_training_data = district_target[district_target['year'] < 2020].reset_index(drop=True)
district_target_testing_data = district_target[district_target['year'] == 2020].reset_index(drop=True)

In [12]:
area_target_training_data = area_target_training_data.drop('year', axis=1)
area_target_testing_data = area_target_testing_data.drop('year', axis=1)
district_target_training_data = district_target_training_data.drop('year', axis=1)
district_target_testing_data = district_target_testing_data.drop('year', axis=1)

## Linear Regression

In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

##### Final Feature Engineering

In [14]:
lr_area_feature_training_data = area_feature_training_data.drop('date_hour', axis=1)
lr_area_feature_testing_data = area_feature_testing_data.drop('date_hour', axis=1)

lr_district_feature_training_data = district_feature_training_data.drop('date_hour', axis=1)
lr_district_feature_testing_data = district_feature_testing_data.drop('date_hour', axis=1)

In [15]:
# target encoding of district/area columns
area_means = area_pre_feature_selection.groupby('area_id')['area_crimes_this_hour'].mean()
district_means = district_pre_feature_selection.groupby('district')['district_crimes_this_hour'].mean()

lr_area_feature_training_data['area_id_target_encoded'] = lr_area_feature_training_data['area_id'].map(area_means)
lr_area_feature_testing_data['area_id_target_encoded'] = lr_area_feature_testing_data['area_id'].map(area_means)

lr_district_feature_training_data['district_target_encoded'] = lr_district_feature_training_data['district'].map(district_means)
lr_district_feature_testing_data['district_target_encoded'] = lr_district_feature_testing_data['district'].map(district_means)

In [16]:
# frequency encoding of district/area columns
area_freq = area_pre_feature_selection['area_id'].value_counts() / len(area_pre_feature_selection)
district_freq = district_pre_feature_selection['district'].value_counts() / len(district_pre_feature_selection)

lr_area_feature_training_data['area_id_freq_encoded'] = lr_area_feature_training_data['area_id'].map(area_freq)
lr_area_feature_testing_data['area_id_freq_encoded'] = lr_area_feature_testing_data['area_id'].map(area_freq)

lr_district_feature_training_data['district_freq_encoded'] = lr_district_feature_training_data['district'].map(district_freq)
lr_district_feature_testing_data['district_freq_encoded'] = lr_district_feature_testing_data['district'].map(district_freq)

In [17]:
lr_area_feature_training_data.drop('area_id', axis=1, inplace=True)
lr_area_feature_testing_data.drop('area_id', axis=1, inplace=True)

lr_district_feature_training_data.drop('district', axis=1, inplace=True)
lr_district_feature_testing_data.drop('district', axis=1, inplace=True)

In [18]:
def patch_datatypes(df):
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype(np.float32)

    int_cols = df.select_dtypes(include=['int64']).columns
    df[int_cols] = df[int_cols].astype(np.int32)    
      
    return df

In [19]:
lr_area_feature_training_data = patch_datatypes(lr_area_feature_training_data)

In [20]:
lr_area_feature_testing_data = patch_datatypes(lr_area_feature_testing_data)

In [21]:
lr_district_feature_training_data = patch_datatypes(lr_district_feature_training_data)

In [22]:
lr_district_feature_testing_data = patch_datatypes(lr_district_feature_testing_data)

In [23]:
def generate_correlation_heatmap(df):
    # Generate a mask to onlyshow the bottom triangle
    mask = np.triu(np.ones_like(df.corr(), dtype=bool))

    # generate heatmap
    plt.figure(figsize=(70,70))
    sns.heatmap(df.corr(), annot=True, mask=mask, vmin=-1, vmax=1)
    plt.title('Correlation Coefficient Of Area Crime Predictors')
    plt.show()

In [ ]:
generate_correlation_heatmap(lr_area_feature_training_data)

In [ ]:
generate_correlation_heatmap(lr_district_feature_training_data)

##### Using VIF to Remove Multicollinearity

In [ ]:
# Function to compute VIF for all features
def compute_vif(feature_df):
    print(f"{datetime.now()} - Starting VIF computation")
    X = feature_df.copy()
    # The calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # Create dataframe to store VIF values
    vif = pd.DataFrame()
    vif["feature"] = X.columns
    vif["vif"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['feature'] != 'intercept']
    
    print(f"{datetime.now()} - Completed VIF computation")
    return vif

In [ ]:
# Function to optimize VIF by dropping features with high VIF values
def optimize_vif(feature_df, vif_threshold):
    print(f"{datetime.now()} - Starting VIF optimization")
    df = feature_df.copy()       

    vif_df = compute_vif(feature_df)
    
    while (vif_df['vif'] >= vif_threshold).any():
        print(f"{datetime.now()} - Current VIF values:\n{vif_df}")
        largest_vif_feature = vif_df.loc[vif_df['vif'].idxmax(), 'feature']
        print(f"{datetime.now()} - Dropping feature: {largest_vif_feature} with VIF score of: {vif_df['vif'].max()}")
        df = df.drop(columns=[largest_vif_feature])
        vif_df = compute_vif(df)
    
    print(f"{datetime.now()} - Completed VIF optimization")
    return vif_df

In [ ]:
lr_area_selected_features_ten = optimize_vif(lr_area_feature_training_data, 10)

In [ ]:
lr_district_selected_features_ten = optimize_vif(lr_district_feature_training_data, 10)

In [ ]:
lr_area_selected_features_five = optimize_vif(lr_area_feature_training_data, 5)

In [ ]:
lr_district_selected_features_five = optimize_vif(lr_district_feature_training_data, 5)

##### Using SFS for Feature Selection

In [ ]:
area_model_ten = LinearRegression()
area_sfs_ten = SFS(area_model_ten, k_features='best', forward=True, floating=False, scoring='neg_mean_squared_error', cv=5, verbose=2)

In [ ]:
area_sfs_ten.fit(lr_area_feature_training_data[[lr_area_selected_features_ten['features'].values]], area_target_training_data)

In [ ]:
area_model_five = LinearRegression()
area_sfs_five = SFS(area_model_five, k_features='best', forward=True, floating=False, scoring='neg_mean_squared_error', cv=5, verbose=2)

In [ ]:
area_sfs_five.fit(lr_area_feature_training_data[[lr_area_selected_features_five['features'].values]], area_target_training_data)

In [ ]:
district_model_ten = LinearRegression()
district_sfs_ten = SFS(district_model_ten, k_features='best', forward=True, floating=False, scoring='neg_mean_squared_error', cv=5, verbose=2)

In [ ]:
district_sfs_ten.fit(lr_district_feature_training_data[[lr_district_selected_features_ten['features'].values]], area_target_training_data)

In [ ]:
district_model_five = LinearRegression()
district_sfs_five = SFS(district_model_five, k_features='best', forward=True, floating=False, scoring='neg_mean_squared_error', cv=5, verbose=2)

In [ ]:
district_sfs_five.fit(lr_district_feature_training_data[[lr_district_selected_features_five['features'].values]], area_target_training_data)

##### Model Training

In [ ]:
# Extract the cross-validation scores from the SFS objects
area_cv_scores_ten = area_sfs_ten.get_metric_dict()
area_cv_scores_five = area_sfs_five.get_metric_dict()

# Get the best scores (least negative mean squared error)
area_best_score_ten = max(area_cv_scores_ten.values(), key=lambda x: x['avg_score'])
area_best_score_five = max(area_cv_scores_five.values(), key=lambda x: x['avg_score'])

# Define the final model based on the best score
area_selected_features = area_sfs_ten.feature_names_ if area_best_score_ten > area_best_score_five else area_sfs_five.feature_names_
lr_area_feature_training_data = lr_area_feature_training_data[[list(area_selected_features)]]
lr_area_feature_testing_data = lr_area_feature_testing_data[[list(area_selected_features)]]

In [ ]:
# Extract the cross-validation scores from the SFS objects
district_cv_scores_ten = district_sfs_ten.get_metric_dict()
district_cv_scores_five = district_sfs_five.get_metric_dict()

# Get the best scores (least negative mean squared error)
district_best_score_ten = max(district_cv_scores_ten.values(), key=lambda x: x['avg_score'])
district_best_score_five = max(district_cv_scores_five.values(), key=lambda x: x['avg_score'])

# Define the final model based on the best score
district_selected_features = district_sfs_ten.feature_names_ if district_best_score_ten > district_best_score_five else district_sfs_five.feature_names_
lr_district_feature_training_data = lr_district_feature_training_data[list(district_selected_features)]
lr_district_feature_testing_data = lr_district_feature_testing_data[list(district_selected_features)]

In [ ]:
# Train the final area model
area_final_lr_model = LinearRegression()
area_final_lr_model.fit(lr_area_feature_training_data, area_target_training_data)

In [ ]:
# Train the final district model
district_final_lr_model = LinearRegression()
district_final_lr_model.fit(lr_district_feature_training_data, district_target_training_data)

##### Model Testing

In [ ]:
# Predict using the area model
area_predictions = area_final_lr_model.predict(lr_area_feature_testing_data)

In [ ]:
# Calculate evaluation metrics for the area model
area_mse = mean_squared_error(area_target_testing_data, area_predictions)
area_rmse = np.sqrt(area_mse)
area_mae = mean_absolute_error(area_target_testing_data, area_predictions)
area_r2 = r2_score(area_target_testing_data, area_predictions)

# Print evaluation metrics for the area model
print("Area Model Performance Metrics:")
print(f"Mean Squared Error (MSE): {area_mse}")
print(f"Root Mean Squared Error (RMSE): {area_rmse}")
print(f"Mean Absolute Error (MAE): {area_mae}")
print(f"R^2 Score: {area_r2}")
print()

In [ ]:
# Predict using the district model
district_predictions = district_final_lr_model.predict(lr_district_feature_testing_data)

In [ ]:
# Calculate evaluation metrics for the district model
district_mse = mean_squared_error(district_target_testing_data, district_predictions)
district_rmse = np.sqrt(district_mse)
district_mae = mean_absolute_error(district_target_testing_data, district_predictions)
district_r2 = r2_score(district_target_testing_data, district_predictions)

# Print evaluation metrics for the district model
print("District Model Performance Metrics:")
print(f"Mean Squared Error (MSE): {district_mse}")
print(f"Root Mean Squared Error (RMSE): {district_rmse}")
print(f"Mean Absolute Error (MAE): {district_mae}")
print(f"R^2 Score: {district_r2}")

## XGBoost

In [24]:
from numpy import sort
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [25]:
xgb_area_feature_training_data = lr_area_feature_training_data.copy()
xgb_area_feature_testing_data = lr_area_feature_testing_data.copy()

xgb_district_feature_training_data = lr_district_feature_training_data.copy()
xgb_district_feature_testing_data = lr_district_feature_testing_data.copy()

##### Training the Area XGBoost Model

In [ ]:
area_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
area_model.fit(xgb_area_feature_training_data, area_target_training_data.values.ravel())

In [ ]:
# Calculate initial accuracy
area_y_pred = area_model.predict(xgb_area_feature_testing_data)
area_accuracy = accuracy_score(area_target_testing_data, area_y_pred)

print(f"Accuracy: {area_accuracy:.4f}")

In [ ]:
area_thresholds = sort(area_model.feature_importances_)
# Initialize variables to store the best results
area_best_accuracy = area_accuracy
area_best_thresh = None
area_best_features = None

In [ ]:
# Iterate over thresholds to find the best feature set
for thresh in area_thresholds:
    # Select features using the threshold
    area_selection = SelectFromModel(area_model, threshold=thresh, prefit=True)
    area_select_X_train = area_selection.transform(xgb_area_feature_training_data)

    # Train the new model with selected features
    area_selection_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
    area_selection_model.fit(area_select_X_train, area_target_training_data)

    # Evaluate the new model
    area_select_X_test = area_selection.transform(xgb_area_feature_testing_data)
    area_predictions = area_selection_model.predict(area_select_X_test)
    area_accuracy = accuracy_score(area_target_testing_data, area_predictions)
    
    # Print the results for the current threshold
    print(f"Thresh={thresh:.3f}, n={area_select_X_train.shape[1]}, Accuracy: {area_accuracy*100.0:.2f}%")
    
    # Update the best accuracy and corresponding features if improved
    if area_accuracy > area_best_accuracy:
        area_best_accuracy = area_accuracy
        area_best_thresh = thresh
        area_best_features = area_selection.get_support(indices=True)

# Print the best threshold and corresponding accuracy
print(f"Best Thresh={area_best_thresh:.3f}, Best Accuracy: {area_best_accuracy*100.0:.2f}%")
print(f"Best Features: {area_best_features}")

In [ ]:
xgb_area_feature_training_data = xgb_area_feature_training_data.iloc[:, area_best_features]
xgb_area_feature_testing_data = xgb_area_feature_testing_data.iloc[:, area_best_features]

##### Training the District XGBoost Model

In [ ]:
district_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
district_model.fit(xgb_district_feature_training_data, district_target_training_data.values.ravel())

In [ ]:
district_y_pred = district_model.predict(xgb_district_feature_testing_data)
district_accuracy = accuracy_score(district_target_testing_data, district_y_pred)
print(f"Accuracy: {district_accuracy:.4f}")

In [ ]:
# Determine thresholds based on feature importances
district_thresholds = np.sort(district_model.feature_importances_)
district_best_accuracy = district_accuracy
district_best_thresh = None
district_best_features = None

In [ ]:
for thresh in district_thresholds:
    # Select features using the threshold
    district_selection = SelectFromModel(district_model, threshold=thresh, prefit=True)
    district_select_X_train = district_selection.transform(xgb_district_feature_training_data)

    # Train the new model with selected features
    district_selection_model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1)
    district_selection_model.fit(district_select_X_train, district_target_training_data)

    # Evaluate the new model
    district_select_X_test = district_selection.transform(xgb_district_feature_testing_data)
    district_predictions = district_selection_model.predict(district_select_X_test)
    district_accuracy = accuracy_score(district_target_testing_data, district_predictions)
    
    # Print the results for the current threshold
    print(f"Thresh={thresh:.3f}, n={district_select_X_train.shape[1]}, Accuracy: {district_accuracy*100.0:.2f}%")
    
    # Update the best accuracy and corresponding features if improved
    if district_accuracy > district_best_accuracy:
        district_best_accuracy = district_accuracy
        district_best_thresh = thresh
        district_best_features = district_selection.get_support(indices=True)

# Print the best threshold and corresponding accuracy
print(f"Best Thresh={district_best_thresh:.3f}, Best Accuracy: {district_best_accuracy*100.0:.2f}%")
print(f"Best Features: {district_best_features}")

In [ ]:
xgb_district_feature_training_data = xgb_district_feature_training_data.iloc[:, district_best_features]
xgb_district_feature_testing_data = xgb_district_feature_testing_data.iloc[:, district_best_features]

##### Hyperparameter Tuning

In [ ]:
# Define the hyperparameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1]
}

In [ ]:
# Create the XGBoost model object
area_xgb_model = xgb.XGBClassifier()

# Create the GridSearchCV object
area_grid_search = GridSearchCV(area_xgb_model, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the training data
area_grid_search.fit(xgb_area_feature_training_data, xgb_area_feature_testing_data.values.ravel())

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", area_grid_search.best_params_)
print("Best score: ", area_grid_search.best_score_)

In [ ]:
# Create the XGBoost model object
district_xgb_model = xgb.XGBClassifier()

# Create the GridSearchCV object
district_grid_search = GridSearchCV(area_xgb_model, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the training data
district_grid_search.fit(xgb_district_feature_training_data, xgb_district_feature_testing_data.values.ravel())

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", district_grid_search.best_params_)
print("Best score: ", district_grid_search.best_score_)

##### Training of Final XGBoost Models

In [ ]:
# Extract the best parameters from the grid search
area_best_params = area_grid_search.best_params_

# Create the final model with the best parameters
area_final_xgb_model = XGBClassifier(**area_best_params)

In [ ]:
# Train the final model with the selected features from the training data
area_final_xgb_model.fit(xgb_area_feature_training_data, area_target_training_data.values.ravel())

In [ ]:
# Evaluate the final model on the test data
area_final_predictions = area_final_xgb_model.predict(xgb_area_feature_testing_data)
area_final_accuracy = accuracy_score(area_target_testing_data, area_final_predictions)
print(f"Final Model Accuracy: {area_final_accuracy:.4f}")

In [ ]:
# Extract the best parameters from the grid search
district_best_params = district_grid_search.best_params_

# Create the final model with the best parameters
district_final_xgb_model = XGBClassifier(**district_best_params)

In [ ]:
# Train the final model with the selected features from the training data
district_final_xgb_model.fit(xgb_district_feature_training_data, district_target_training_data.values.ravel())

In [ ]:
# Evaluate the final model on the test data
district_final_predictions = district_final_xgb_model.predict(xgb_district_feature_testing_data)
district_final_accuracy = accuracy_score(district_target_testing_data, district_final_predictions)
print(f"Final Model Accuracy: {district_final_accuracy:.4f}")

## Random Forest

In [26]:
from sklearn.ensemble import RandomForestRegressor

In [27]:
rf_area_feature_training_data = lr_area_feature_training_data.copy()
rf_area_feature_testing_data = lr_area_feature_testing_data.copy()

rf_district_feature_training_data = lr_district_feature_training_data.copy()
rf_district_feature_testing_data = lr_district_feature_testing_data.copy()

##### Training Initial Area RF Model

In [ ]:
rf_area_model = RandomForestRegressor(verbose=2)
rf_area_model.fit(rf_area_feature_training_data, area_target_training_data)

In [ ]:
area_acc_initial = rf_area_model.score(rf_area_feature_testing_data, area_target_testing_data)
print(f'Accuracy before feature selection: {area_acc_initial:.4f}')

In [ ]:
area_importances = rf_area_model.feature_importances_
area_feature_names = area_feature_training_data.columns

area_feature_importance_df = pd.DataFrame({'feature':area_feature_names, 'importance':area_importances})
area_feature_importance_df.sort_values(by='importance', ascending=False, inplace=True)

In [ ]:
for idx, row in area_feature_importance_df.iterrows():
    print(row['feature'], '- importance:', row['importance'])

In [ ]:
area_cumulative_importance = np.cumsum(area_feature_importance_df['importance'])
area_threshold = 0.95
area_selected_features = area_feature_importance_df['feature'][area_cumulative_importance <= area_threshold]

rf_area_feature_training_data = rf_area_feature_training_data[area_selected_features]
rf_area_feature_testing_data = rf_area_feature_testing_data[area_selected_features]

##### Training initial District RF Model

In [ ]:
rf_district_model = RandomForestRegressor(verbose=2)
rf_district_model.fit(rf_district_feature_training_data, district_target_training_data)

In [ ]:
district_acc_initial = rf_district_model.score(rf_district_feature_testing_data, district_target_testing_data)
print(f'Accuracy before feature selection: {district_acc_initial:.4f}')

In [ ]:
district_importances = rf_district_model.feature_importances_
district_feature_names = district_feature_training_data.columns

district_feature_importance_df = pd.DataFrame({'feature':district_feature_names, 'importance':district_importances})
district_feature_importance_df.sort_values(by='importance', ascending=False, inplace=True)

In [ ]:
for idx, row in district_feature_importance_df.iterrows():
    print(row['feature'], '- importance:', row['importance'])

In [ ]:
district_cumulative_importance = np.cumsum(district_feature_importance_df['importance'])
district_threshold = 0.95
district_selected_features = district_feature_importance_df['feature'][district_cumulative_importance <= district_threshold]

rf_district_feature_training_data = rf_district_feature_training_data[district_selected_features]
rf_district_feature_testing_data = rf_district_feature_testing_data[district_selected_features]

##### Hyperparameter Tuning

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [ ]:
rf_area_model = RandomForestRegressor(random_state=42)
grid_search_area = GridSearchCV(estimator=rf_area_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='r2')

In [ ]:
grid_search_area.fit(rf_area_feature_training_data, area_target_training_data)

In [ ]:
best_params_area = grid_search_area.best_params_
best_score_area = grid_search_area.best_score_
print(f'Best Parameters for Area Model: {best_params_area}')
print(f'Best R2 Score for Area Model: {best_score_area:.4f}')

In [ ]:
rf_district_model = RandomForestRegressor(random_state=42)
grid_search_district = GridSearchCV(estimator=rf_district_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='r2')

In [ ]:
grid_search_district.fit(rf_district_feature_training_data, district_target_training_data)

In [ ]:
best_params_district = grid_search_district.best_params_
best_score_district = grid_search_district.best_score_
print(f'Best Parameters for District Model: {best_params_district}')
print(f'Best R2 Score for District Model: {best_score_district:.4f}')

##### Training the Final RF Models

In [ ]:
final_rf_area_model = grid_search_area.best_estimator_
final_rf_area_model.fit(rf_area_feature_training_data, area_target_training_data)

In [ ]:
area_acc_final = final_rf_area_model.score(rf_area_feature_testing_data, area_target_testing_data)
print(f'Final Accuracy for Area Model after Hyperparameter Tuning: {area_acc_final:.4f}')

In [ ]:
final_rf_district_model = grid_search_district.best_estimator_
final_rf_district_model.fit(rf_district_feature_training_data, district_target_training_data)

In [ ]:
district_acc_final = final_rf_district_model.score(rf_district_feature_testing_data, district_target_testing_data)
print(f'Final Accuracy for District Model after Hyperparameter Tuning: {district_acc_final:.4f}')